In [1]:
import math
import numpy as np
from IPython.display import clear_output
from tqdm import tqdm_notebook as tqdm

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.color_palette("bright")
import matplotlib as mpl
import matplotlib.cm as cm

import torch
from torch import Tensor
from torch import nn
from torch.nn  import functional as F 
from torch.autograd import Variable
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

import time
import random

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
def save_checkpoint(epoch, model, optimizer):
    """
    Save model checkpoint.
    :param epoch: epoch number
    :param model: model
    :param optimizer: optimizer
    """
    state = {'epoch': epoch,
             'model': model,
             'optimizer': optimizer}
    filename = 'checkpoint_resnet18_cutmix_cifar100.pth.tar'
    torch.save(state, filename)

class AverageMeter(object):
    """
    Keeps track of most recent, average, sum, and count of a metric.
    """

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [3]:
# 数据集读取

epochs = 200
pre_epoch = 0
BATCH_SIZE = 128
LR = 0.01
mean = [0.5070751592371323, 0.48654887331495095, 0.4409178433670343]
std = [0.2673342858792401, 0.2564384629170883, 0.27615047132568404]
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

train_dataset = torchvision.datasets.CIFAR100(root='data', train=True, download=False, transform=transform_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

test_dataset = torchvision.datasets.CIFAR100(root='data', train=False, download=False, transform=transform_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=2)

In [4]:
class ResidualBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(ResidualBlock, self).__init__()
        self.left = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel, outchannel, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outchannel)
            )

    def forward(self, x):
        out = self.left(x)
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, ResidualBlock, num_classes=100):
        super(ResNet, self).__init__()
        self.inchannel = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        self.layer1 = self.make_layer(ResidualBlock, 64,  2, stride=1)
        self.layer2 = self.make_layer(ResidualBlock, 128, 2, stride=2)
        self.layer3 = self.make_layer(ResidualBlock, 256, 2, stride=2)
        self.layer4 = self.make_layer(ResidualBlock, 512, 2, stride=2)
        self.fc = nn.Linear(512, num_classes)

    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)   #strides=[1,1]
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)    #核为4，步长为4
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


def ResNet18():

    return ResNet(ResidualBlock)

In [5]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    """1.论文里的公式2，求出B的rw,rh"""
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)
 
    # uniform
    """2.论文里的公式2，求出B的rx,ry（bbox的中心点）"""
    cx = np.random.randint(W)
    cy = np.random.randint(H)
    #限制坐标区域不超过样本大小
 
    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)
    """3.返回剪裁B区域的坐标值"""
    return bbx1, bby1, bbx2, bby2


In [6]:
checkpoint = 'checkpoint_resnet18_cutmix_cifar100.pth.tar'
print_freq = 20

def main():
    """
    Training.
    """
    
    global start_epoch, classes, epoch, checkpoint
    
    # 初始化模型
    
    if checkpoint is None:
        start_epoch = 0
        model = ResNet18()
        optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)
    else:
        checkpoint = torch.load(checkpoint, map_location = 'cuda')
        start_epoch = checkpoint['epoch'] + 1
        print('\nLoaded checkpoint from epoch %d.\n' % start_epoch)
        model = checkpoint['model']
        optimizer = checkpoint['optimizer']
    
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    
    for epoch in range(start_epoch, epochs):
        
        if epoch == 100:
            optimizer = optim.SGD(model.parameters(), lr=LR*0.1, momentum=0.9, weight_decay=5e-4)
        if epoch == 150:
            optimizer = optim.SGD(model.parameters(), lr=LR*0.01, momentum=0.9, weight_decay=5e-4)
        
        train(train_loader = train_loader,
             model = model,
             criterion=criterion,
             optimizer=optimizer,
             epoch=epoch)
        save_checkpoint(epoch, model, optimizer)
        evaluate(test_loader, model)
        

def train(train_loader, model, criterion, optimizer, epoch):
    
    model = model.train()
    
    batch_time = AverageMeter()  # forward prop. + back prop. time
    data_time = AverageMeter()  # data loading time
    losses = AverageMeter()  # loss

    start = time.time()
    total = 0
    correct = 0
    alpha = 1.0
    for i, data in enumerate(train_loader, 0):
        length = len(train_loader)
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        
        if random.random()<0.5:
            lam = np.random.beta(alpha, alpha)
            rand_index = torch.randperm(inputs.size()[0]).to(device)
            labels_a = labels
            labels_b = labels[rand_index]
            bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), lam)
            inputs[:, :, bbx1:bbx2, bby1:bby2] = inputs[rand_index, :, bbx1:bbx2, bby1:bby2]
            lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (inputs.size()[-1] * inputs.size()[-2]))
        else:
            labels_a = labels
            labels_b = labels
            lam = 1.0
        optimizer.zero_grad()

        # forward + backward
        outputs = model(inputs)
        loss = criterion(outputs, labels_a)*lam + criterion(outputs, labels_b) * (1. - lam)
        loss.backward()
        optimizer.step()
        
        losses.update(loss.item(), inputs.size(0))
        batch_time.update(time.time() - start)
        start = time.time()
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += predicted.eq(labels.data).cpu().sum()
        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Batch Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data Time {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'.format(epoch, i, len(train_loader),
                                                    batch_time=batch_time,
                                                    data_time=data_time, loss=losses))
            f1 = open("train_resnet18_cutmix_cifar100.txt", "a")
            f1.write('Epoch: [{0}][{1}/{2}]\t'
                  'Batch Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data Time {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t\n'.format(epoch, i, len(train_loader),
                                                    batch_time=batch_time,
                                                    data_time=data_time, loss=losses))
            f1.close()
    print(correct/total)
    f2 = open("train_acc_resnet18_cutmix_cifar100.txt", "a")
    f2.write('resnet_cutmix_cifar100测试分类准确率为：%.3f%%\n' % (100 * correct / total))
    f2.close()
def evaluate(test_loader, model):
    
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for i, data in enumerate(test_loader, 0):
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            # 取得分最高的那个类 (outputs.data的索引号)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum()
        print('测试分类准确率为：%.3f%%' % (100 * correct / total))
        f3 = open("test_acc_resnet18_cifar100.txt", "a")
        f3.write('resnet_cutmix_cifar100测试分类准确率为：%.3f%%\n' % (100 * correct / total))
        f3.close()
        acc = 100. * correct / total
        best_acc = 85
        if acc > best_acc:
            f3 = open("best_acc_resnet18_cutmix_cifar100.txt", "w")
            f3.write("EPOCH=%d,best_acc= %.3f%%" % (epoch + 1, acc))
            f3.close()
            best_acc = acc
if __name__ == '__main__':
    main()


Loaded checkpoint from epoch 86.



<ipython-input-5-9aec3f1bf01e>:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  cut_w = np.int(W * cut_rat)
<ipython-input-5-9aec3f1bf01e>:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more detai

Epoch: [86][0/391]	Batch Time 5.548 (5.548)	Data Time 0.000 (0.000)	Loss 1.8626 (1.8626)	
Epoch: [86][20/391]	Batch Time 0.172 (0.431)	Data Time 0.000 (0.000)	Loss 0.1387 (0.5611)	
Epoch: [86][40/391]	Batch Time 0.172 (0.306)	Data Time 0.000 (0.000)	Loss 2.6596 (0.8310)	
Epoch: [86][60/391]	Batch Time 0.172 (0.263)	Data Time 0.000 (0.000)	Loss 0.2809 (0.8279)	
Epoch: [86][80/391]	Batch Time 0.172 (0.242)	Data Time 0.000 (0.000)	Loss 2.6181 (0.9987)	
Epoch: [86][100/391]	Batch Time 0.172 (0.228)	Data Time 0.000 (0.000)	Loss 0.1674 (0.9663)	
Epoch: [86][120/391]	Batch Time 0.172 (0.219)	Data Time 0.000 (0.000)	Loss 0.1564 (0.9304)	
Epoch: [86][140/391]	Batch Time 0.172 (0.213)	Data Time 0.000 (0.000)	Loss 0.1346 (0.9029)	
Epoch: [86][160/391]	Batch Time 0.172 (0.208)	Data Time 0.000 (0.000)	Loss 2.2453 (0.9006)	
Epoch: [86][180/391]	Batch Time 0.172 (0.205)	Data Time 0.000 (0.000)	Loss 0.1526 (0.9267)	
Epoch: [86][200/391]	Batch Time 0.172 (0.202)	Data Time 0.000 (0.000)	Loss 2.0663 (0.9

Epoch: [90][160/391]	Batch Time 0.172 (0.194)	Data Time 0.000 (0.000)	Loss 2.5299 (1.1595)	
Epoch: [90][180/391]	Batch Time 0.172 (0.193)	Data Time 0.000 (0.000)	Loss 0.1888 (1.1382)	
Epoch: [90][200/391]	Batch Time 0.187 (0.191)	Data Time 0.000 (0.000)	Loss 2.6039 (1.1591)	
Epoch: [90][220/391]	Batch Time 0.172 (0.190)	Data Time 0.000 (0.000)	Loss 2.1051 (1.1634)	
Epoch: [90][240/391]	Batch Time 0.187 (0.189)	Data Time 0.000 (0.000)	Loss 1.5492 (1.1580)	
Epoch: [90][260/391]	Batch Time 0.187 (0.188)	Data Time 0.000 (0.000)	Loss 2.5773 (1.1552)	
Epoch: [90][280/391]	Batch Time 0.172 (0.187)	Data Time 0.000 (0.000)	Loss 0.2283 (1.1227)	
Epoch: [90][300/391]	Batch Time 0.172 (0.187)	Data Time 0.000 (0.000)	Loss 0.2025 (1.1370)	
Epoch: [90][320/391]	Batch Time 0.187 (0.186)	Data Time 0.000 (0.000)	Loss 3.0336 (1.1263)	
Epoch: [90][340/391]	Batch Time 0.172 (0.186)	Data Time 0.000 (0.000)	Loss 0.1939 (1.1115)	
Epoch: [90][360/391]	Batch Time 0.172 (0.185)	Data Time 0.000 (0.000)	Loss 0.619

Epoch: [94][320/391]	Batch Time 0.187 (0.187)	Data Time 0.000 (0.000)	Loss 2.3676 (1.0849)	
Epoch: [94][340/391]	Batch Time 0.172 (0.186)	Data Time 0.000 (0.000)	Loss 0.1844 (1.0649)	
Epoch: [94][360/391]	Batch Time 0.203 (0.186)	Data Time 0.000 (0.000)	Loss 2.7417 (1.0625)	
Epoch: [94][380/391]	Batch Time 0.187 (0.186)	Data Time 0.000 (0.000)	Loss 2.1148 (1.0745)	
tensor(0.7768)
测试分类准确率为：71.460%
Epoch: [95][0/391]	Batch Time 2.888 (2.888)	Data Time 0.000 (0.000)	Loss 1.4282 (1.4282)	
Epoch: [95][20/391]	Batch Time 0.171 (0.308)	Data Time 0.000 (0.000)	Loss 2.6616 (1.6422)	
Epoch: [95][40/391]	Batch Time 0.187 (0.245)	Data Time 0.000 (0.000)	Loss 1.1404 (1.4910)	
Epoch: [95][60/391]	Batch Time 0.187 (0.223)	Data Time 0.000 (0.000)	Loss 1.3453 (1.2441)	
Epoch: [95][80/391]	Batch Time 0.187 (0.212)	Data Time 0.000 (0.000)	Loss 0.1855 (1.2021)	
Epoch: [95][100/391]	Batch Time 0.187 (0.205)	Data Time 0.000 (0.000)	Loss 0.1296 (1.1945)	
Epoch: [95][120/391]	Batch Time 0.172 (0.201)	Data Tim

...

Epoch: [181][380/391]	Batch Time 0.172 (0.185)	Data Time 0.000 (0.000)	Loss 1.7530 (0.7563)	
tensor(0.8432)
测试分类准确率为：78.610%
Epoch: [182][0/391]	Batch Time 2.685 (2.685)	Data Time 0.000 (0.000)	Loss 2.0889 (2.0889)	
Epoch: [182][20/391]	Batch Time 0.172 (0.298)	Data Time 0.000 (0.000)	Loss 0.4136 (0.9648)	
Epoch: [182][40/391]	Batch Time 0.187 (0.239)	Data Time 0.000 (0.000)	Loss 1.9852 (0.8604)	
Epoch: [182][60/391]	Batch Time 0.187 (0.219)	Data Time 0.000 (0.000)	Loss 0.0173 (0.8388)	
Epoch: [182][80/391]	Batch Time 0.187 (0.209)	Data Time 0.000 (0.000)	Loss 1.5200 (0.7751)	
Epoch: [182][100/391]	Batch Time 0.187 (0.202)	Data Time 0.000 (0.000)	Loss 0.5707 (0.7638)	
Epoch: [182][120/391]	Batch Time 0.187 (0.198)	Data Time 0.000 (0.000)	Loss 0.0153 (0.7215)	
Epoch: [182][140/391]	Batch Time 0.172 (0.195)	Data Time 0.000 (0.000)	Loss 0.0144 (0.7183)	
Epoch: [182][160/391]	Batch Time 0.172 (0.193)	Data Time 0.000 (0.000)	Loss 1.5925 (0.7323)	
Epoch: [182][180/391]	Batch Time 0.172 (0.19

Epoch: [186][120/391]	Batch Time 0.172 (0.199)	Data Time 0.000 (0.000)	Loss 1.0292 (0.6878)	
Epoch: [186][140/391]	Batch Time 0.172 (0.196)	Data Time 0.000 (0.000)	Loss 1.6098 (0.6914)	
Epoch: [186][160/391]	Batch Time 0.187 (0.194)	Data Time 0.000 (0.000)	Loss 2.0339 (0.7051)	
Epoch: [186][180/391]	Batch Time 0.187 (0.192)	Data Time 0.000 (0.000)	Loss 1.2088 (0.7227)	
Epoch: [186][200/391]	Batch Time 0.187 (0.191)	Data Time 0.000 (0.000)	Loss 0.0220 (0.7211)	
Epoch: [186][220/391]	Batch Time 0.172 (0.190)	Data Time 0.000 (0.000)	Loss 0.0117 (0.7312)	
Epoch: [186][240/391]	Batch Time 0.172 (0.189)	Data Time 0.000 (0.000)	Loss 0.0148 (0.7405)	
Epoch: [186][260/391]	Batch Time 0.187 (0.188)	Data Time 0.000 (0.000)	Loss 1.0837 (0.7150)	
Epoch: [186][280/391]	Batch Time 0.187 (0.187)	Data Time 0.000 (0.000)	Loss 0.0435 (0.6784)	
Epoch: [186][300/391]	Batch Time 0.172 (0.187)	Data Time 0.000 (0.000)	Loss 0.0140 (0.7019)	
Epoch: [186][320/391]	Batch Time 0.172 (0.186)	Data Time 0.000 (0.000)

Epoch: [190][260/391]	Batch Time 0.172 (0.188)	Data Time 0.000 (0.000)	Loss 1.7352 (0.7469)	
Epoch: [190][280/391]	Batch Time 0.172 (0.187)	Data Time 0.000 (0.000)	Loss 1.6746 (0.7517)	
Epoch: [190][300/391]	Batch Time 0.187 (0.187)	Data Time 0.000 (0.000)	Loss 0.0134 (0.7710)	
Epoch: [190][320/391]	Batch Time 0.179 (0.186)	Data Time 0.000 (0.000)	Loss 1.8628 (0.7736)	
Epoch: [190][340/391]	Batch Time 0.187 (0.186)	Data Time 0.000 (0.000)	Loss 0.0160 (0.7629)	
Epoch: [190][360/391]	Batch Time 0.172 (0.185)	Data Time 0.000 (0.000)	Loss 1.1599 (0.7661)	
Epoch: [190][380/391]	Batch Time 0.187 (0.185)	Data Time 0.000 (0.000)	Loss 0.0239 (0.7612)	
tensor(0.8515)
测试分类准确率为：78.550%
Epoch: [191][0/391]	Batch Time 2.703 (2.703)	Data Time 0.000 (0.000)	Loss 0.0166 (0.0166)	
Epoch: [191][20/391]	Batch Time 0.172 (0.298)	Data Time 0.000 (0.000)	Loss 0.0186 (0.7724)	
Epoch: [191][40/391]	Batch Time 0.187 (0.240)	Data Time 0.000 (0.000)	Loss 2.0099 (0.8195)	
Epoch: [191][60/391]	Batch Time 0.172 (0.2

tensor(0.8522)
测试分类准确率为：78.620%
Epoch: [195][0/391]	Batch Time 2.706 (2.706)	Data Time 0.000 (0.000)	Loss 2.3867 (2.3867)	
Epoch: [195][20/391]	Batch Time 0.172 (0.298)	Data Time 0.000 (0.000)	Loss 0.0153 (0.6525)	
Epoch: [195][40/391]	Batch Time 0.172 (0.240)	Data Time 0.000 (0.000)	Loss 1.9344 (0.8935)	
Epoch: [195][60/391]	Batch Time 0.172 (0.219)	Data Time 0.000 (0.000)	Loss 0.3881 (0.8021)	
Epoch: [195][80/391]	Batch Time 0.172 (0.209)	Data Time 0.000 (0.000)	Loss 0.0192 (0.8253)	
Epoch: [195][100/391]	Batch Time 0.172 (0.203)	Data Time 0.000 (0.000)	Loss 1.7434 (0.8419)	
Epoch: [195][120/391]	Batch Time 0.187 (0.199)	Data Time 0.000 (0.000)	Loss 2.0851 (0.8637)	
Epoch: [195][140/391]	Batch Time 0.187 (0.196)	Data Time 0.000 (0.000)	Loss 0.0135 (0.8278)	
Epoch: [195][160/391]	Batch Time 0.187 (0.194)	Data Time 0.000 (0.000)	Loss 2.0612 (0.8063)	
Epoch: [195][180/391]	Batch Time 0.187 (0.193)	Data Time 0.000 (0.000)	Loss 1.1048 (0.8445)	
Epoch: [195][200/391]	Batch Time 0.172 (0.19

Epoch: [199][140/391]	Batch Time 0.187 (0.196)	Data Time 0.000 (0.000)	Loss 0.0108 (0.7036)	
Epoch: [199][160/391]	Batch Time 0.172 (0.194)	Data Time 0.000 (0.000)	Loss 0.0125 (0.6865)	
Epoch: [199][180/391]	Batch Time 0.187 (0.192)	Data Time 0.000 (0.000)	Loss 1.4723 (0.7388)	
Epoch: [199][200/391]	Batch Time 0.187 (0.191)	Data Time 0.000 (0.000)	Loss 2.0201 (0.7631)	
Epoch: [199][220/391]	Batch Time 0.172 (0.190)	Data Time 0.000 (0.000)	Loss 0.0182 (0.7675)	
Epoch: [199][240/391]	Batch Time 0.187 (0.189)	Data Time 0.000 (0.000)	Loss 0.3896 (0.7772)	
Epoch: [199][260/391]	Batch Time 0.172 (0.188)	Data Time 0.000 (0.000)	Loss 0.0169 (0.7806)	
Epoch: [199][280/391]	Batch Time 0.187 (0.188)	Data Time 0.000 (0.000)	Loss 0.0164 (0.7705)	
Epoch: [199][300/391]	Batch Time 0.172 (0.187)	Data Time 0.000 (0.000)	Loss 0.0163 (0.7657)	
Epoch: [199][320/391]	Batch Time 0.172 (0.187)	Data Time 0.000 (0.000)	Loss 1.5131 (0.7726)	
Epoch: [199][340/391]	Batch Time 0.187 (0.186)	Data Time 0.000 (0.000)